# Text generation with an RNN

Adapted from the [tensorflow example](https://www.tensorflow.org/tutorials/sequences/text_generation), to run on [datahub.ucsd.edu](datahub.ucsd.edu), with TF 1.14. 

Robert Twomey, rtwomey@ucsd.edu.

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly to seuquentially add characters.

The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Goals

- Understand the evolution of samples during training, and the relationship between the number of training ephocs and the quality of the generated output. 
- Produce (or download) your own textual corpus to experiment with parameters, training, and results. In [his article](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) cited above, Karpathy plays with sources including Shakespeare, Wikipedia articles, Algebraic Geometry (Latex), and Linux source code, and baby names.
- Experiment with:
  - The effect of the number of training epochs
  - The parameters, training, and results. 
  - Add additional RNN layers
- Share your results with class.

## Background

"Neural Nets" are loosely inspired on neurobiology. Now people call them units, nodes, etc., as opposed to "neurons". Each unit has a number of inputs (with weights), an activation function, a bias (offset), and an output. These weights and biases are the trainable parameters of the network that can be "learned".

![A_neuron_a_simplified_mathematical_model](images/neuronal_model.png)

Neural Networks are interconnected layers of these kinds of neuronal units: 

<!--![Layered_net](images/layers.png)-->
<img src="images/layers.png" width=400px>

Each unit has an activation function, a bias, and ever connection going in or out of the node has a weight. All of these are trainable parameters of the network. For the image above, we have 9 neurons, with 32 connections (each with weight), 9 biases. This makes for 41 trainable parameters). This diagram is a fully connected network (each node connects to every node on the next layer). We call this a feed-forward network, because input data (at the left) is propogated / "feeds forward" from node to node in layer to layer, producing outputs (at the right).

Complicated feed forward mechanisms (like this MNIST handwritten digit classifier, below) might have 13k trainable parameters. These are the parameters (weights and biases) of the network that are learned during training. The RNN model we use below has 4M trainable parameters.

<!--![Mnist](images/mnist.png)-->
<img src="images/mnist.png" width=400px>

Today we are working with Recurrent Neural Networks (RNNs). They are a special kind of network where the system feed data back into itself (see below). We evaluate them sequentially: computing values, then passing information forward to the network to evaluate the next item in a sequence. They are very prevalent for modeling sequences of data, whether text, or time series like drawings or music. 

<!--![RNN Unit](images/rnn_unit.png)-->
<img src="images/rnn_unrolled.png" width=600px> 

## Setup

### Import TensorFlow and other libraries

In [64]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# tf.enable_eager_execution()

import numpy as np
import os
import time
from IPython.display import Image

tf.enable_eager_execution()

### Use Loveletter Dataset

In [65]:
# Use your own file:
path_to_file = "love_total.txt"

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [66]:
# Default example training on Shakespeare:
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# Use your own file:
# path_to_file = "try_with_your_own_file.txt"

# Robert's file
# path_to_file = "script.txt"




### Read the data

First, look in the text.

In [67]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding = "ISO-8859-1")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 299308 characters


In [68]:
# Take a look at the first 250 characters in text
print(text[:250])

Just being around you gives me a happiness that no one else could ever give me. You have given me a love that I had never known before .I love being with you and cannot imagine a world without you in it. Each morning, I wake up and thank God that you


In [69]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

80 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [70]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [71]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '#' :   3,
  '&' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '1' :  11,
  '3' :  12,
  ':' :  13,
  ';' :  14,
  '?' :  15,
  'A' :  16,
  'B' :  17,
  'C' :  18,
  'D' :  19,
  ...
}


In [72]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'Just being ar' ---- characters mapped to int ---- > [25 60 58 59  1 41 44 48 53 46  1 40 57]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text. 

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right. 

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [73]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

J
u
s
t
 


The `batch` method lets us easily convert these individual characters to sequences of the desired size. 

In [74]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Just being around you gives me a happiness that no one else could ever give me. You have given me a l'
'ove that I had never known before .I love being with you and cannot imagine a world without you in it'
'. Each morning, I wake up and thank God that you have been a part of my life. I just cannot find the '
'words to tell you how happy I am that we met and got together. You have been the rock I turned to dur'
'ing difficulties and the guiding light that lit the way whenever I was lost. I want to tell you that '


For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [75]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [76]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Just  being around you gives me a happiness that no one else could ever give me. You have given me a '
Target data: 'ust being around you gives me a happiness that no one else could ever give me. You have given me a l'


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the `RNN` considers the previous step context in addition to the current input character.

In [77]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 25 ('J')
  expected output: 60 ('u')
Step    1
  input: 60 ('u')
  expected output: 58 ('s')
Step    2
  input: 58 ('s')
  expected output: 59 ('t')
Step    3
  input: 59 ('t')
  expected output: 1 (' ')
Step    4
  input: 1 (' ')
  expected output: 41 ('b')


### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [78]:
# Batch size (number of 100 word sequences to train on)
BATCH_SIZE = 64

# How many individual training steps per samples (steps)
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

### Neuronal Modal


__NOTE__: Insert an image here diagramming basic neural net

In [79]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Next define a function to build the model.

Use `CuDNNGRU` if running on GPU.  

In [80]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  rnn2 = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [81]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),

    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [82]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-liklihood of the next character:

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [83]:
for input_example_batch, target_example_batch in dataset.take(1): 
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 80) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length: 

In [84]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           20480     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (64, None, 1024)          3938304   
_________________________________________________________________
dense_3 (Dense)              (64, None, 80)            82000     
Total params: 4,040,784
Trainable params: 4,040,784
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary. 

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [85]:
# sampled_indices = tf.random.multinomial(example_batch_predictions[0], num_samples=1) # TF 1.12
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [86]:
sampled_indices

array([35,  1, 18, 49, 63, 39, 27, 50, 65, 77, 34,  4, 61, 58, 11, 57, 23,
       79, 19, 37, 13,  8, 35, 50, 47, 38, 29, 74, 34, 13, 30, 44, 79, 78,
       24, 36, 27, 34, 65, 49,  6, 29, 62, 12, 43, 37, 23, 11, 77, 41, 18,
       74, 47, 61, 10, 56, 58, 75, 12, 43, 79, 59, 33, 68, 61, 48,  8, 63,
       34, 16, 52, 18, 35, 51, 11, 21, 42, 29, 11,  2,  9,  3, 19, 23, 60,
       69,  2, 62, 60, 41,  4, 68, 46, 38, 64,  6, 69, 24, 19, 30])

Decode these to see the text predicted by this untrained model:

In [87]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "king about you--\nFeeling happy and slightly insane.\n\nNow I hope I meet that same wizard,\nFor I'd giv"

Next Char Predictions: 
 'T CjxYLkz¹S&vs1rHâDV:,TkhWN¡S:OeâÃIULSzj(Nw3dVH1¹bC¡hv.qs¦3dâtR\x93vi,xSAmCTl1FcN1!-#DHu\x94!wub&\x93gWy(\x94IDO'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_softmax_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions. 

Because our model returns logits, we need to set the `from_logits` flag. 


In [88]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 80)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3822503


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.train.AdamOptimizer` with default arguments and the loss function.

NOTE TO SELF: Insert reference about optimizers, gradient descent, and Adam particularly.

In [89]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [90]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 3 epochs to train the model. Increase the number of epochs to increase training time.

In [91]:
EPOCHS=50

In [92]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/50


W1019 20:57:56.855441 139883499505472 util.py:244] Unresolved object in checkpoint: (root).optimizer
W1019 20:57:56.856829 139883499505472 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer
W1019 20:57:56.858124 139883499505472 util.py:244] Unresolved object in checkpoint: (root).optimizer.global_step
W1019 20:57:56.858875 139883499505472 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta1_power
W1019 20:57:56.859582 139883499505472 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta2_power
W1019 20:57:56.860257 139883499505472 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-0.embeddings
W1019 20:57:56.860932 139883499505472 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-1.kernel
W1019 20:57:56.861656 139883499505472 util.py:244] Unresolved object in checkpoint: (root).optimizer.

46/46 [==============================] - 3s 59ms/step - loss: 3.5181
Epoch 2/50
46/46 [==============================] - 2s 42ms/step - loss: 2.4533
Epoch 3/50
46/46 [==============================] - 2s 42ms/step - loss: 2.2241
Epoch 4/50
46/46 [==============================] - 2s 42ms/step - loss: 2.0716
Epoch 5/50
46/46 [==============================] - 2s 42ms/step - loss: 1.9394
Epoch 6/50
46/46 [==============================] - 2s 42ms/step - loss: 1.8238
Epoch 7/50
46/46 [==============================] - 2s 41ms/step - loss: 1.7174
Epoch 8/50
46/46 [==============================] - 2s 41ms/step - loss: 1.6145
Epoch 9/50
46/46 [==============================] - 2s 43ms/step - loss: 1.5151
Epoch 10/50
46/46 [==============================] - 2s 43ms/step - loss: 1.4109
Epoch 11/50
46/46 [==============================] - 2s 42ms/step - loss: 1.3032
Epoch 12/50
46/46 [==============================] - 2s 42ms/step - loss: 1.1887
Epoch 13/50
46/46 [=============================

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built. 

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [93]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training/ckpt_50'

In [94]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [95]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            20480     
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 80)             82000     
Total params: 4,040,784
Trainable params: 4,040,784
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a multinomial distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one word. After predicting the next word, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted words.


![To generate text the model's output is fed back to the input](https://tensorflow.org/tutorials/sequences/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [96]:

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing) 
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  
  for i in range(num_generate):
      predictions = model(input_eval)
        
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [110]:
print(generate_text(model, start_string=u"You"))


You're r a cheerful countenance at all times
and give else I do,
Onetiful.
My thoughts of you are like raindrops the whole drfor many way deep within.

It must be the power of light, but each time
leaves in wind revealed themselves,

the retreating shadow of a fox, daybreak.
We expect y, love you meant for all smooth lips can say,
Have given their hearts upon no person but you.
When you are in the room my thoughts never fly out of window: you always concentrate my whole senses.

In that word, beautiful in all languages, but most so in yours--Amor mio-¦ing through the ring means falling through the spaces between the objects that together make the ring.

On the way through, clay don't teach you at school. They don't teach you how to love somebody. They don't teach you how to love somebody. They don't teach you how to love somebody...
Beautiful.
My thoughts of you are like raindrops on flowers...
Beautiful.
My thoughts of you are like raindrops on flowers...
Beautiful.
My thoughts of you

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

# Activities

1. Gradually increase the number of training epochs, and see how your generated output evolves. "Sample" the model a number of times and store your results.
2. Train on your own corpus, instead of Shakespeare.
  - Find a plain text data source and upload it to datahub.
  - How long does your text need to be? (shakespeare is 1.1M characters)
3. Experiment with the network parameters. In ML we call these "hyperparameters", parameters of the network, to distinguish from the internal weights and biases ("parameters") that are learned during training.
  - How does changing ```temperature = 1.0``` change the qualities of the generated text? E.g. try values 0 < 1.0, > 1.0, near 0.0
  - How does changing the number of embedding dimensions ```embedding_dim = 256``` change the results? This is more important in word-level models.
  - How does changing the number of RNN units (internal "nodes" in the LSTM) change the results? By default we have ```rnn_units = 1024```
  - Does changing the length of training sequences ```seq_length = 100``` change the results? 
  - Add a new layer to your network (copy the rnn statement in the model construction)
4. Future directions: 
  - Move on to a word-level RNN model (like this https://github.com/hunkim/word-rnn-tensorflow)
  - Move on to fine-tuning the GPT-2 model (https://github.com/nshepperd/gpt-2, https://pmbaumgartner.github.io/blog/gpt2-jokes/)